In [2]:
import ROOT
from ROOT import gStyle
import numpy as np
import ctypes
import os
import pandas as pd

def biner(o_bin,m_bin,f_bin,f_step,l_step):
    a=np.arange(o_bin,m_bin,f_step)
    b=np.arange(m_bin,f_bin+l_step,l_step)
    return np.concatenate((a,b))

def normalization(hist_list,norm_bin):
    for hist in hist_list:
        for i in range(1,hist.GetNbinsX()+2):
            value=hist.GetBinContent(i)
            error=hist.GetBinError(i)
            sf=hist.GetBinWidth(i)/norm_bin
            hist.SetBinContent(i,value/sf)
            hist.SetBinError(i,error/sf)

In [3]:
##### SELECTING GENERATOR FOR SIGNAL SAMPLE
signal_sample="Signal_Sherpa.root"
ztt_sample="Ztautau_Sherpa.root"
type_of_mc = ztt_sample.strip('.root').split('_')[1]
#### CREATING THE CORRESPONDING MC.ROOT FILE EVERYTIME YOU CHANGE THE SIGNAL SAMPLE

os.system("rm MC.root")
os.system("hadd MC.root "+"BG.root "+signal_sample)

0

hadd Target file: MC.root
hadd compression setting for all output: 1
hadd Source file 1: BG.root
hadd Source file 2: Signal_Sherpa.root
hadd Target path: MC.root:/


In [4]:
histos_no_rebin={

"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],
    
}
histos_rebin={
"delta_phi":[0.4,0.4,0.4],
"lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[0.0,0.5,0.5],
"tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[0.0,0.5,0.5],
"delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_y":[1.0,1.0,1.0],
"omega":[1,0.5,0.5],
"rnn_score_1p":[0.2,0.1,0.1],
"rnn_score_3p":[0.8,0.2,0.2],
"lep_pt":[120,20,20],
"lep_pt_basic_dphi_drap_btag_iso":[120,10,10],
"lep_pt_basic_dphi_drap_btag_iso_rnn":[120,10,10],
"tau_pt":[120,20,20],
"ljet0_pt":[150,50,50],
"ljet1_pt":[150,50,50],
"pt_bal":[0.2,0.02,0.04],
"Z_centrality":[1,0.1,0.5],
"reco_mass_i":[120,10,20],
"reco_mass_o":[120,10,20],
"reco_mass_":[120,10,20],
"mass_jj":[1500,250,500],
}
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for i in histos:
    print(i)
    ROOT.TH1.SetDefaultSumw2()
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Ztautau=ROOT.TFile.Open(ztt_sample,"READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Zjet.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")
    Higgs=ROOT.TFile.Open("Higgs.root","READ")
    Signal=ROOT.TFile.Open(signal_sample,"READ")

    a=i
    
    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    higgs=Higgs.Get(a)
    ztt=Ztautau.Get(a)
    
    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    higgs.SetDirectory(0)
    ztt.SetDirectory(0)
    
    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    Higgs.Close()
    Ztautau.Close()

    hs = ROOT.THStack("hs","")
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(s,histos[i][0],e,histos[i][1],histos[i][2])
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"signal",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        higgs=higgs.Rebin(nb,"higgs",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        ztt=ztt.Rebin(nb,"ztt",rebining)
        
        hist_list=[signal,data,vv,tt,t,z,w,mc,ztt,higgs]
        normalization(hist_list,histos[i][1])
    
    signal.SetFillColor(ROOT.kOrange+7)
    signal.SetLineWidth(0)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    higgs.SetFillColor(ROOT.kWhite)
    higgs.SetLineColor(ROOT.kWhite)
    data.SetLineColor(ROOT.kBlack)
    ztt.SetLineColor(ROOT.kViolet)
    ztt.SetFillColor(ROOT.kViolet)
    data.SetMarkerStyle(20)
    data.SetMarkerSize(0.5)



    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(higgs)
    hs.Add(ztt)
    hs.Add(signal)
    
    monte_carlo=signal.Clone()
    monte_carlo.SetLineWidth(1)
    monte_carlo.Add(z,1)
    monte_carlo.Add(tt,1)
    monte_carlo.Add(vv,1)
    monte_carlo.Add(w,1)
    monte_carlo.Add(t,1)
    monte_carlo.Add(higgs,1)
    monte_carlo.Add(ztt,1)
    
    ratio=monte_carlo.Clone()
    ratio.Divide(data)
    
    ratio_sg_mc=signal.Clone()
    ratio_sg_mc.Divide(mc)
    
    gStyle.SetOptStat(1111111)
    gStyle.SetStatY(0.97);                
    gStyle.SetStatX(1.0);
    gStyle.SetStatW(0.12);                
    gStyle.SetStatH(0.12);

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()


    data.Draw("pe")
    hs.Draw("HIST same")
    data.Draw("pe same")
    #if "rnn_score" in i:
    #pad1.SetLogy()


    data . GetYaxis (). SetRangeUser (0 ,1.3*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s,e)
    if i in histos_rebin:
        data . GetYaxis (). SetTitle ("Events/"+str(histos[i][1])+" GeV")
    
        
    legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( higgs ," Higgs ")
    legend . AddEntry ( ztt ," Ztt")
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (0)
    legend . Draw ()
    

    hs.SetTitle(a)

    data.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.9,0.7,"Z#rightarrow #tau#mu")
    
    max_ratio = ratio.GetMaximum()
    if max_ratio > 3:
        max_ratio = 3
    if max_ratio < 1.5:
        max_ratio = 1.5

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.15 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.0)
    pad2.Draw ()
    pad2.cd ()
    ratio.SetFillColorAlpha(ROOT.kBlue,0.35)
    ratio.Draw ("E2")
    ratio.SetTitle("")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (ratio.GetMinimum() ,max_ratio)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    #ratio . GetXaxis (). SetTitle (histos[i][3])
    ratio . GetXaxis (). SetTitleSize (0.00)
    ratio.GetXaxis().SetLabelSize(0.00)
    ratio.GetYaxis().SetLabelSize(0.08)
    ratio.SetMarkerStyle(8)
    ratio.SetMarkerSize(0.6)

        

    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    
    separators = []
    resolution = 10 # In percentage
    i=0
    range_sep = []
    while i < (int(max_ratio)+1):
        step = resolution/100
        if i > ratio.GetMinimum():
            range_sep.append(i)
        i = i + step
     
    for i in range (len(range_sep)):
        sep = ROOT.TLine(s ,range_sep[i],e,range_sep[i])
        sep.SetLineColor(ROOT.kBlack)
        sep.SetLineWidth(1)
        sep.SetLineStyle(2)
        separators.append(sep)
        
    line . Draw (" same ")   
    for i in separators:
        i.Draw("same")
    
    canvas3.cd()
    pad3 = ROOT . TPad (" pad3","pad3" ,0 ,0.00 ,1 ,0.15)
    pad3.SetRightMargin(0.03)
    pad3.SetLeftMargin(0.08)
    pad3.SetTopMargin(0)
    pad3.SetBottomMargin(0.1)
    pad3.Draw ()
    pad3.cd ()
    ratio_sg_mc.SetStats(0)
    ratio_sg_mc . GetYaxis (). SetRangeUser (0.0 ,1.02)
    ratio_sg_mc . GetXaxis (). SetRangeUser (s ,e)
    ratio_sg_mc . GetYaxis (). SetTitle ("SIGNAL/MC")
    ratio_sg_mc . GetYaxis (). SetTitleSize (0.15)
    ratio_sg_mc . GetYaxis (). SetTitleOffset (0.25)
    ratio_sg_mc.GetYaxis().SetLabelSize(0.09)
    ratio_sg_mc.GetXaxis().SetLabelSize(0.12)
    ratio_sg_mc.SetMarkerStyle(8)
    ratio_sg_mc.SetMarkerSize(0.6)
    
    ratio_sg_mc.Draw ("hist p")
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line11 = ROOT . TLine (s ,0.80 ,e,0.80)
    line11 . SetLineColor ( ROOT . kBlack )
    line11 . SetLineWidth (1)
    line11 . SetLineStyle(2)
    line12 = ROOT . TLine (s ,0.60 ,e,0.60)
    line12 . SetLineColor ( ROOT . kBlack )
    line12 . SetLineWidth (1)
    line12 . SetLineStyle(2)
    line13 = ROOT . TLine (s ,0.40 ,e,0.40)
    line13 . SetLineColor ( ROOT . kBlack )
    line13 . SetLineWidth (1)
    line13 . SetLineStyle(2)
    line14 = ROOT . TLine (s ,0.20 ,e,0.20)
    line14 . SetLineColor ( ROOT . kBlack )
    line14 . SetLineWidth (1)
    line14 . SetLineStyle(2)
    sline = ROOT . TLine (s ,1 ,e,1)
    sline . SetLineColor ( ROOT . kBlack )
    sline . SetLineWidth (2)
    sline.Draw("same")
    line11 . Draw (" same ")
    line12 . Draw (" same ")
    line13 . Draw (" same ")
    line14 . Draw (" same ")
        







    canvas3.Print(a+"_"+type_of_mc+".pdf")
    
    
    if a=="tau_nprongs_cuts_tpt":
        mc_yield_1p=mc.Integral(1,2)
        data_yield_1p=data.Integral(1,2)
        print("\n"*2)
        print("MC 1p= ",round(mc_yield_1p,3))
        print("DATA 1p= ",round(data_yield_1p,3))
        print("\n"*2)
        mc_yield_3p=mc.Integral(3,4)
        data_yield_3p=data.Integral(3,4)
        print("\n"*2)
        print("MC 3p= ",round(mc_yield_3p,3))
        print("DATA 3p= ",round(data_yield_3p,3))
        print("\n"*2)

n_bjets
lepiso
n_jets_interval
delta_phi
lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_y
omega
rnn_score_1p
rnn_score_3p
lep_pt
lep_pt_basic_dphi_drap_btag_iso
lep_pt_basic_dphi_drap_btag_iso_rnn
tau_pt
ljet0_pt
ljet1_pt
pt_bal
Z_centrality
reco_mass_i
reco_mass_o
reco_mass_
mass_jj


Info in <TCanvas::Print>: pdf file n_bjets_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lepiso_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_jets_interval_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_phi_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt_Sherpa.pdf has been created
Warning in <TCanvas::Constructor>:

In [9]:
histos_no_rebin={

"n_bjets":[],
"lepiso":[],
"n_jets_interval":[],
    
}
histos_rebin={
"delta_phi":[0.4,0.4,0.4],
"lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[0.0,0.5,0.5],
"tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[0.0,0.5,0.5],
"delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt":[1,0.5,0.5],
"delta_y":[1.0,1.0,1.0],
"omega":[1,0.5,0.5],
"rnn_score_1p":[0.2,0.1,0.1],
"rnn_score_3p":[0.8,0.2,0.2],
"lep_pt":[120,20,20],
"tau_pt":[120,20,20],
"ljet0_pt":[150,50,50],
"ljet1_pt":[150,50,50],
"pt_bal":[0.2,0.02,0.04],
"Z_centrality":[1,0.1,0.5],
"reco_mass_i":[120,10,20],
"reco_mass_o":[120,10,20],
"reco_mass_":[120,10,20],
"mass_jj":[1500,250,500],
}
histos={}
histos.update(histos_no_rebin)
histos.update(histos_rebin)
    
    
for i in histos:
    print(i)
    MC=ROOT.TFile.Open("MC.root","READ")
    Data = ROOT.TFile.Open("Data.root","READ")
    Ztautau=ROOT.TFile.Open(ztt_sample,"READ")
    VV=ROOT.TFile.Open("VV.root","READ")
    Wjets=ROOT.TFile.Open("Wjets.root","READ")
    Zjets=ROOT.TFile.Open("Zjet.root","READ")
    Ttbar=ROOT.TFile.Open("ttbar.root","READ")
    SingleTop=ROOT.TFile.Open("singletop.root","READ")
    Higgs=ROOT.TFile.Open("Higgs.root","READ")
    Signal=ROOT.TFile.Open(signal_sample,"READ")

    a=i
    
    mc=MC.Get(a)
    data=Data.Get(a)
    signal=Signal.Get(a)
    vv=VV.Get(a)
    w=Wjets.Get(a)
    z=Zjets.Get(a)
    tt=Ttbar.Get(a)
    t=SingleTop.Get(a)
    higgs=Higgs.Get(a)
    ztt=Ztautau.Get(a)
    
    mc.SetDirectory(0)
    signal.SetDirectory(0)
    data.SetDirectory(0)
    vv.SetDirectory(0)
    w.SetDirectory(0)
    z.SetDirectory(0)
    tt.SetDirectory(0)
    t.SetDirectory(0)
    higgs.SetDirectory(0)
    ztt.SetDirectory(0)
    
    MC.Close()
    Signal.Close()
    Data.Close()
    VV.Close()
    Wjets.Close()
    Zjets.Close()
    Ttbar.Close()
    SingleTop.Close()
    Higgs.Close()
    Ztautau.Close()

    hs = ROOT.THStack("hs","")
    s=data.GetXaxis().GetBinLowEdge(1)
    e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
    if i in histos_rebin:
        ob=data.GetNbinsX()
        s=data.GetXaxis().GetBinLowEdge(1)
        e=data.GetXaxis().GetBinUpEdge(data.GetNbinsX())
        rebining=biner(s,histos[i][0],e,histos[i][1],histos[i][2])
        nb=len(rebining)-1
        mc=mc.Rebin(nb,"signal",rebining)
        signal=signal.Rebin(nb,"signal",rebining)
        data=data.Rebin(nb,"data",rebining)
        vv=vv.Rebin(nb,"vv",rebining)
        tt=tt.Rebin(nb,"tt",rebining)
        t=t.Rebin(nb,"t",rebining)
        higgs=higgs.Rebin(nb,"higgs",rebining)
        z=z.Rebin(nb,"z",rebining)
        w=w.Rebin(nb,"w",rebining)
        ztt=ztt.Rebin(nb,"ztt",rebining)
        
        if histos[i][1]!=histos[i][2]:
            hist_list=[signal,data,vv,tt,t,z,w,mc,ztt,higgs]
            normalization(hist_list,histos[i][1])
    
    signal.SetFillColor(ROOT.kOrange+7)
    signal.SetLineWidth(0)
    w.SetFillColor(ROOT.kGreen)
    w.SetLineColor(ROOT.kGreen)
    z.SetFillColor(ROOT.kRed)
    z.SetLineColor(ROOT.kRed)
    vv.SetFillColor(ROOT.kBlue)
    vv.SetLineColor(ROOT.kBlue)
    tt.SetFillColor(ROOT.kYellow)
    tt.SetLineColor(ROOT.kYellow)
    t.SetFillColor(ROOT.kCyan)
    t.SetLineColor(ROOT.kCyan)
    higgs.SetFillColor(ROOT.kWhite)
    higgs.SetLineColor(ROOT.kWhite)
    data.SetLineColor(ROOT.kBlack)
    ztt.SetLineColor(ROOT.kViolet)
    ztt.SetFillColor(ROOT.kViolet)
    data.SetMarkerStyle(20)
    data.SetMarkerSize(0.5)



    hs.Add(vv)
    hs.Add(w)
    hs.Add(z)
    hs.Add(tt)
    hs.Add(t)
    hs.Add(higgs)
    hs.Add(ztt)
    hs.Add(signal)
    
    #################### SCALING FACTORS FROM FIT ####################
        
    signal.Scale(0.982)
    ztt.Scale(1.016)
        
    ##################################################################
    
    monte_carlo=signal.Clone()
    monte_carlo.SetLineWidth(1)
    monte_carlo.Add(vv,1)
    monte_carlo.Add(w,1)
    monte_carlo.Add(z,1)
    monte_carlo.Add(tt,1)
    monte_carlo.Add(t,1)
    monte_carlo.Add(higgs,1)
    monte_carlo.Add(ztt,1)

    ratio=monte_carlo.Clone()
    ratio.Divide(data)
    
    ratio_sg_mc=signal.Clone()
    ratio_sg_mc.Divide(mc)

    gStyle.SetOptStat(1111111)
    gStyle.SetStatY(0.97);                
    gStyle.SetStatX(1.0);
    gStyle.SetStatW(0.12);                
    gStyle.SetStatH(0.12);

    canvas3 = ROOT.TCanvas("canvas3")
    canvas3.cd()

    pad1 = ROOT . TPad (" pad1 "," pad1 " ,0 ,0.35 ,1 ,1)
    pad1.SetTopMargin(0.03)
    pad1.SetRightMargin(0.03)
    pad1.SetLeftMargin(0.08)
    pad1.SetBottomMargin(0.0)
    pad1.Draw ()
    pad1.cd ()


    data.Draw("pe")
    hs.Draw("HIST same")
    data.Draw("pe same")
    #if "rnn_score" in i:
    pad1.SetLogy()


    data . GetYaxis (). SetRangeUser (0.1 ,10.3*data.GetBinContent(data.GetMaximumBin()))
    data . GetXaxis (). SetRangeUser (s,e)
    
        
    legend = ROOT . TLegend (0.45 ,0.80 ,0.85 ,0.95)
    legend . AddEntry ( vv ,"Diboson")
    legend . AddEntry ( data ," Data ")
    legend . AddEntry ( signal ," Signal ")
    legend . AddEntry ( w ," W+jets ")
    legend . AddEntry ( z ," Z+jets ")
    legend . AddEntry ( t ," Single top ")
    legend . AddEntry ( tt ," ttbar ")
    legend . AddEntry ( higgs ," Higgs ")
    legend . AddEntry ( ztt ," Ztt")
    legend.SetNColumns(3)
    gStyle.SetLegendBorderSize(0)
    legend . SetLineWidth (0)
    legend . Draw ()
    

    hs.SetTitle(a)

    data.SetTitle("")
    l=ROOT.TLatex()
    l.SetNDC ()
    l.DrawLatex(0.9,0.7,"Z#rightarrow #tau#mu")

    canvas3.cd()
    pad2 = ROOT . TPad (" pad2 "," pad2 " ,0 ,0.15 ,1 ,0.35)
    pad2.SetRightMargin(0.03)
    pad2.SetLeftMargin(0.08)
    pad2.SetTopMargin(0)
    pad2.SetBottomMargin(0.0)
    pad2.Draw ()
    pad2.cd ()
    ratio.Draw ("pe")
    ratio.SetTitle("")
    ratio.SetStats(0)
    ratio . GetYaxis (). SetRangeUser (0.4,1.6)
    ratio . GetXaxis (). SetRangeUser (s ,e)
    ratio . GetYaxis (). SetTitle ("MC/DATA")
    ratio . GetYaxis (). SetTitleSize (0.15)
    ratio . GetYaxis (). SetTitleOffset (0.25)
    #ratio . GetXaxis (). SetTitle (histos[i][3])
    ratio . GetXaxis (). SetTitleSize (0.00)
    ratio.GetXaxis().SetLabelSize(0.00)
    ratio.GetYaxis().SetLabelSize(0.08)

        

    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line = ROOT . TLine (s ,1 ,e,1)
    line . SetLineColor ( ROOT . kBlack )
    line . SetLineWidth (2)
    line2 = ROOT . TLine (s ,1.1 ,e,1.1)
    line2 . SetLineColor ( ROOT . kBlack )
    line2 . SetLineWidth (1)
    line2 . SetLineStyle(2)
    line3 = ROOT . TLine (s ,0.9 ,e,0.9)
    line3 . SetLineColor ( ROOT . kBlack )
    line3 . SetLineWidth (1)
    line3 . SetLineStyle(2)
    line4 = ROOT . TLine (s ,1.2 ,e,1.2)
    line4 . SetLineColor ( ROOT . kBlack )
    line4 . SetLineWidth (1)
    line4 . SetLineStyle(2)
    line5 = ROOT . TLine (s ,0.8 ,e,0.8)
    line5 . SetLineColor ( ROOT . kBlack )
    line5 . SetLineWidth (1)
    line5 . SetLineStyle(2)
    line6 = ROOT . TLine (s ,1.3 ,e,1.3)
    line6 . SetLineColor ( ROOT . kBlack )
    line6 . SetLineWidth (1)
    line6 . SetLineStyle(2)
    line7 = ROOT . TLine (s ,0.7 ,e,0.7)
    line7 . SetLineColor ( ROOT . kBlack )
    line7 . SetLineWidth (1)
    line7 . SetLineStyle(2)
    line8 = ROOT . TLine (s ,1.4 ,e,1.4)
    line8 . SetLineColor ( ROOT . kBlack )
    line8 . SetLineWidth (1)
    line8 . SetLineStyle(2)
    line9 = ROOT . TLine (s ,0.6 ,e,0.6)
    line9 . SetLineColor ( ROOT . kBlack )
    line9 . SetLineWidth (1)
    line9 . SetLineStyle(2)
    line . Draw (" same ")
    line2 . Draw (" same ")
    line3 . Draw (" same ")
    line4 . Draw (" same ")
    line5 . Draw (" same ")
    line6 . Draw (" same ")
    line7 . Draw (" same ")
    line8 . Draw (" same ")
    line9 . Draw (" same ")
    line . Draw (" same ")
    
    canvas3.cd()
    pad3 = ROOT . TPad (" pad3","pad3" ,0 ,0.00 ,1 ,0.15)
    pad3.SetRightMargin(0.03)
    pad3.SetLeftMargin(0.08)
    pad3.SetTopMargin(0)
    pad3.SetBottomMargin(0.1)
    pad3.Draw ()
    pad3.cd ()
    ratio_sg_mc.SetStats(0)
    ratio_sg_mc . GetYaxis (). SetRangeUser (0.0 ,1.02)
    ratio_sg_mc . GetXaxis (). SetRangeUser (s ,e)
    ratio_sg_mc . GetYaxis (). SetTitle ("BG/MC")
    ratio_sg_mc . GetYaxis (). SetTitleSize (0.15)
    ratio_sg_mc . GetYaxis (). SetTitleOffset (0.25)
    ratio_sg_mc.GetYaxis().SetLabelSize(0.09)
    ratio_sg_mc.GetXaxis().SetLabelSize(0.12)
    ratio_sg_mc.SetMarkerStyle(8)
    ratio_sg_mc.SetMarkerSize(0.6)
    
    ratio_sg_mc.Draw ("hist p")
    
    ###### SETTING ALL THE HORIZONTAL DASHED LINES #######
    
    line11 = ROOT . TLine (s ,0.80 ,e,0.80)
    line11 . SetLineColor ( ROOT . kBlack )
    line11 . SetLineWidth (1)
    line11 . SetLineStyle(2)
    line12 = ROOT . TLine (s ,0.60 ,e,0.60)
    line12 . SetLineColor ( ROOT . kBlack )
    line12 . SetLineWidth (1)
    line12 . SetLineStyle(2)
    line13 = ROOT . TLine (s ,0.40 ,e,0.40)
    line13 . SetLineColor ( ROOT . kBlack )
    line13 . SetLineWidth (1)
    line13 . SetLineStyle(2)
    line14 = ROOT . TLine (s ,0.20 ,e,0.20)
    line14 . SetLineColor ( ROOT . kBlack )
    line14 . SetLineWidth (1)
    line14 . SetLineStyle(2)
    sline = ROOT . TLine (s ,1 ,e,1)
    sline . SetLineColor ( ROOT . kBlack )
    sline . SetLineWidth (2)
    sline.Draw("same")
    line11 . Draw (" same ")
    line12 . Draw (" same ")
    line13 . Draw (" same ")
    line14 . Draw (" same ")
        







    canvas3.Print(a+"_AfterFit.pdf")
    
    
    if a=="tau_nprongs_cuts_tpt":
        mc_yield_1p=mc.Integral(1,2)
        data_yield_1p=data.Integral(1,2)
        print("\n"*2)
        print("MC 1p= ",round(mc_yield_1p,3))
        print("DATA 1p= ",round(data_yield_1p,3))
        print("\n"*2)
        mc_yield_3p=mc.Integral(3,4)
        data_yield_3p=data.Integral(3,4)
        print("\n"*2)
        print("MC 3p= ",round(mc_yield_3p,3))
        print("DATA 3p= ",round(data_yield_3p,3))
        print("\n"*2)

n_bjets
lepiso
n_jets_interval
delta_phi
lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taulep_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_lepjet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_R_taujet_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt
delta_y
omega
rnn_score_1p
rnn_score_3p
lep_pt
tau_pt
ljet0_pt
ljet1_pt
pt_bal
Z_centrality
reco_mass_i
reco_mass_o
reco_mass_
mass_jj


Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_bjets_AfterFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lepiso_AfterFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file n_jets_interval_AfterFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file delta_phi_AfterFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file lep_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nji_zcen_omega_mreco_tpt_AfterFit.pdf has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: canvas3
Info in <TCanvas::Print>: pdf file tau_eta_basic_dphi_drap_btag_iso_rnn_ptl_j1pt_j2pt_ptbal_mjj_nj

### PLOT SINGULAR PROPERTIES

In [ ]:

File="Wjets.root"
histo="tau_matched"

Signal=ROOT.TFile.Open(File,"READ")
signal=Signal.Get(histo)
canvas4=ROOT.TCanvas("canvas4")
canvas4.cd()
#signal=signal.Rebin(50,"signal",rebining)
signal.Draw("HIST")
signal.SetFillColor(ROOT.kGreen)
signal.SetLineColor(ROOT.kGreen)
signal.SetTitle(histo)

legend = ROOT.TLegend (0.5 ,0.5 ,0.85 ,0.55)
legend.AddEntry ( signal ,File[:-5])
legend.Draw("same")

canvas4.Print(File[:-5]+"_"+histo+".pdf")

### PLOTTING BDT SCORE

In [ ]:
canvas5=ROOT.TCanvas("canvas5")
canvas5.cd()
w.Draw("HIST")
z.Draw("HIST,same")
vv.Draw("HIST,same")
tt.Draw("HIST,same")
t.Draw("HIST,same")
signal.Draw("HIST,same")


w.SetLineColor(ROOT.kGreen)

z.SetLineColor(ROOT.kRed)

vv.SetLineColor(ROOT.kBlue)

tt.SetLineColor(ROOT.kYellow)

t.SetLineColor(ROOT.kCyan)

legend.Draw()

canvas5.SetLogy()

canvas5.Print("BDT_score.pdf")



In [ ]:
rebining

In [ ]:
MC=ROOT.TFile.Open("MC.root","READ")
Signal=ROOT.TFile.Open("Signal.root","READ")
VV=ROOT.TFile.Open("VV.root","READ")
Wjets=ROOT.TFile.Open("Wjets.root","READ")
Zjets=ROOT.TFile.Open("Zjet.root","READ")
Ttbar=ROOT.TFile.Open("ttbar.root","READ")
SingleTop=ROOT.TFile.Open("singletop.root","READ")

gStyle.SetOptStat(1111111)
a="reco_mass_jetn_btag_iso_bdt_tmass_omega"

mc=MC.Get(a)
signal=Signal.Get(a)
vv=VV.Get(a)
w=Wjets.Get(a)
z=Zjets.Get(a)
tt=Ttbar.Get(a)
t=SingleTop.Get(a)

mc.SetDirectory(0)
signal.SetDirectory(0)
vv.SetDirectory(0)
w.SetDirectory(0)
z.SetDirectory(0)
tt.SetDirectory(0)
t.SetDirectory(0)

MC.Close()
Signal.Close()
VV.Close()
Wjets.Close()
Zjets.Close()
Ttbar.Close()
SingleTop.Close()

hs = ROOT.THStack("hs","")
#'''
ob=300
s=0
e=300
n=6
nb=int(ob/n)
rebining=np.arange(s,e+n*(e-s)/ob,n*(e-s)/ob)
mc=mc.Rebin(nb,"signal",rebining)
signal=signal.Rebin(nb,"signal",rebining)
#data=data.Rebin(nb,"data",rebining)
vv=vv.Rebin(nb,"vv",rebining)
tt=tt.Rebin(nb,"tt",rebining)
t=t.Rebin(nb,"t",rebining)
z=z.Rebin(nb,"z",rebining)
w=w.Rebin(nb,"w",rebining)
#'''

w.SetFillColor(ROOT.kGreen)
w.SetLineColor(ROOT.kGreen)
z.SetFillColor(ROOT.kRed)
z.SetLineColor(ROOT.kRed)
vv.SetFillColor(ROOT.kBlue)
vv.SetLineColor(ROOT.kBlue)
tt.SetFillColor(ROOT.kYellow)
tt.SetLineColor(ROOT.kYellow)
t.SetFillColor(ROOT.kCyan)
t.SetLineColor(ROOT.kCyan)




hs.Add(vv)
hs.Add(w)
hs.Add(z)
hs.Add(tt)
hs.Add(t)
hs.Add(signal)




canvas6 = ROOT.TCanvas("canvas8")
canvas6.cd()


hs.Draw("HIST")

legend = ROOT . TLegend (0.8 ,0.4 ,0.9 ,0.6)
legend . AddEntry ( vv ,"Diboson")
legend . AddEntry ( signal ," Signal ")
legend . AddEntry ( w ," W+jets ")
legend . AddEntry ( z ," Z+jets ")
legend . AddEntry ( t ," Single top ")
legend . AddEntry ( tt ," ttbar ")

legend . SetLineWidth (0)
legend . Draw ()



hs.SetTitle(a)





canvas6.Print(a+".pdf")
